In [17]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem.porter import *

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        try:
            yield eval(l)
        except:
            continue

In [4]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [5]:
dataset = []
for d in readGz("renttherunway_final_data.json.gz"):
    dataset.append(d)


In [6]:
len(dataset)

192462

In [7]:
def heightConversion(h):
    ft, inch = h.split('\' ')
    ft = int(ft)
    inch = int(inch.replace('\"', ''))
    return ft * 12 + inch

In [8]:
new_dataset = []
punctuation = set(string.punctuation)

for d in dataset:
    feature_needed = ['weight', 'height', 'fit', 'size', 'body type', 'review_text', 'review_summary', 'rating']
    keys = list(d.keys())
    if all([i in keys for i in feature_needed]):
        d['weight'] = int(d['weight'].replace("lbs", ""))
        d['height'] = heightConversion(d['height'])
        d['review_text'] = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
        d['review_summary'] = ''.join([c for c in d['review_summary'].lower() if not c in punctuation])
        new_dataset.append(d)

In [18]:

wordCount = defaultdict(int)
stemmer = PorterStemmer()
sw = stopwords.words("English")
for d in dataset:
    for w in d['review_text'].split():
        w = stemmer.stem(w)
        wordCount[w] += 1

len(wordCount)

63250

In [ ]:
counts = [(wordCount[w],w) for w in wordCount if w not in sw]
counts.sort()
counts.reverse()

In [10]:
len(new_dataset)

153441

In [11]:
train_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15
trainlen = int(len(new_dataset) * train_ratio)
validlen = int(len(new_dataset) * validation_ratio)
testlen = int(len(new_dataset) * test_ratio)
train = new_dataset[:trainlen]
valid = new_dataset[trainlen:(trainlen+validlen)]
test = new_dataset[trainlen+validlen:]

In [12]:
ytrain = [d['fit'] for d in train]
yvalid = [d['fit'] for d in valid]
ytest = [d['fit'] for d in test]

In [13]:
Xtrain = [[1, d['weight'], d['height'], d['size']] for d in train]
Xvalid = [[1, d['weight'], d['height'], d['size']] for d in valid]
Xtest = [[1, d['weight'], d['height'], d['size']] for d in test]

In [14]:
def accuracy(pred, y):
    correct = pred == y
    return sum(correct)/len(y)

In [16]:
# Baseline Model - using only weight and height
baselineMod = linear_model.LogisticRegression(C = 1)
baselineMod.fit(Xtrain,ytrain)
blPred = baselineMod.predict(Xvalid)
blacc = accuracy(blPred, yvalid)
blacc
# blacc, pd.Series(ytrain).value_counts(), set(blPred)

0.7363138686131386

In [ ]:
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

In [ ]:
words = [x[1] for x in counts[:1000]]

In [ ]:
def allFeat(d):
    feat = [1]
    return feat

In [17]:
blacc

0.7377042057698991